# Analyzing Regular Text

After I have checked all codes were correclty and outcomes were prcise in the development environment, part of Software Engineering was applied which is OOP aka Object Orientaed Programming which abbreviates all codes in the develpement enverinment and put them in a proffisional way and more systimatic as well as more easier in updating and knowing mistakes.

## Load data

In [1]:
import numpy as np
import pandas as pd

In [2]:
class ConsultationAnaysis:
    '''create multiple objects which have consultations details'''
    def __init__(self, file ):
        self.consultations = self._create_dataframe(file)
        self.series_consult_without_null_vals = self._ignore_null_values(self.consultations)
        self.consult_lst = self._split_text(self.series_consult_without_null_vals)
        self.consults_number = self._compute_number_consultations(self.consult_lst)
        self.len_consutlation_lst = self._compute_length(self.consult_lst)
        self.date_time_lst = self._extract_dates_times(self.consult_lst)
        self.duration = self._compute_durations(self.date_time_lst)
        self.unique_no = self._unique_of_length_or_duration_stat(self.len_consutlation_lst)
        self.duration_stats = self._unique_of_length_or_duration_stat(self.duration)
        
    def _create_dataframe(self,file):
        '''create consultations dataframe'''
        df = pd.read_excel(file)
        consult = df.loc[:,['CONSULTATIONS']]
        return consult
    
    def _ignore_null_values(self,df):
        '''remove null values'''
        df = df.iloc[:,0]
        mask = df.isna()
        series_consult = df[~mask]
        return series_consult
        
    def _split_text(self,series):
        '''split each consultation '''
        series_lsts = series.apply(lambda x: x.split(';'))
        return series_lsts
    
    def _compute_number_consultations(self, series):
        '''compute number of consultations for each patient'''
        series_number = series.apply(lambda x: len(x)).value_counts()
        consults_number = pd.DataFrame(series_number).rename(columns={'CONSULTATIONS':'Frquent'})
        return consults_number
    
    def _compute_length(self,series):
        '''find length of each consultation'''
        len_text_lst = series.apply(lambda x: [len(n) for n in x])
        return len_text_lst
    
    def _unique_of_length_or_duration_stat(self, series):
        '''find number of consultations with and without Reply date time or duration statistics'''
        total_lst = []
        for lst in series:
            total_lst.extend(lst)
            
        if str(series) != str(self.duration):    
            len_without_reply = len([n for n in total_lst if n==31])
            len_with_reply = len(total_lst) - len_without_reply

            without_with_reply_details = pd.DataFrame(
                                [set(total_lst),[len_without_reply,len_with_reply]],
                                columns=['Without Reply', 'With Reply'],
                                index = ['Length of Text','Number of consultations']
                                )
            return without_with_reply_details
        
        else:
            return pd.DataFrame(pd.Series(total_lst).describe(), columns=['Statistics'])
    
    def _extract_dates_times(self, series):
        '''extract dates times'''
        series_date_time = series.apply(lambda x: [ x[i][8:25] +'  '+ x[i][31:] for i in range(len(x)) ])
        return series_date_time
    
    def _compute_durations(self, series):
        '''compute durations by subtract Reply date time from Request date time'''
        series_durations = series.apply(
            lambda x: [
                pd.to_datetime(x[i][19:], errors='coerce',dayfirst=True) - \
                pd.to_datetime(x[i][:16], errors='coerce', dayfirst=True) \
                for i in range(len(x))
            ])
        return series_durations

In [3]:
# passing path of data to class and store it in varialbe
consults = ConsultationAnaysis('big project.xlsx',)

### How many patients who did or did not receive medical consulations?

In [4]:
print(f'There were {consults.consultations.isna().sum()[0]} patients had not received consultations.')

There were 18448 patients had not received consultations.


In [5]:
print(f'There were {consults.series_consult_without_null_vals.shape[0]} patient had recieved consultations.')

There were 11508 patient had recieved consultations.


### Did there any consultatios without reply? If yes, how many did have?

In [6]:
consults.unique_no

,Without Reply,With Reply
Length of Text,31,47
Number of consultations,1208,12720


### How many were total consultations?

In [7]:
print(f'There were {consults.unique_no.iloc[1,:].sum()} consultations')

There were 13928 consultations


### How long consumeed consultation or the duration betweeen the request and reply?

In [8]:
consults.duration_stats

,Statistics
count,12720
mean,0 days 01:57:56.316037735
std,0 days 08:25:37.195638597
min,0 days 00:00:00
25%,0 days 00:32:00
50%,0 days 01:02:00
75%,0 days 01:53:00
max,15 days 16:01:00
